In [7]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import psycopg2 as pg2
import math
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import datetime

In [10]:
class SecondPCLAnalysis:
    
    def __init__(self, password):
        self.password = password

    def data_from_redshift(self, file_name):
        #Getting the original data
        print("[INFO] Establisging connection with AWS")
        conn = pg2.connect(dbname='edwreplica',
                           user='rjangada@upgrade.com',
                           password=self.password,
                           host='edwreplica.ci0plcdfijlw.us-west-2.redshift.amazonaws.com')
        print("[INFO] Connection successful")
        
        query = ("""
        With cte as(
        select b.ssn_hash, lir.id, 
        convert_timezone('PST',lir.create_date::date)::date as create_date,
        lir.product_type 
        from loanreview.loan_in_review as lir 
        left join funnel.loan_application as lap
        on lir.id = lap.id
        left join funnel.borrower as b
        on lap.borrower_id = b.id
        where lir.loan_status in('OPENED','ISSUED') 
        order by b.ssn_hash, convert_timezone('PST',lir.create_date::date)::date)

        select * ssn_hash, id, create_date, product_type
        from cte
                """
                )
        print("[INFO] Running SQL query")
        raw_data = pd.read_sql_query(sql=query, con=conn)
        print("[INFO] Data received from Redshift")
        raw_data.to_csv(file_name)
        conn.close()
        return
    
    def file_splitter(self, csv_file):

        total_rows = sum(1 for row in (open(csv_file)))
        batch_size = 1000000
        file_list = []
        print("[INFO] Splitting the data from Redshift")
        
        
        df = pd.read_csv(csv_file)
        columns = list(df.head(0))
        #columns=columns[1:]
        

        for i in range(1, total_rows, batch_size):
            df = pd.read_csv(csv_file, header=0, nrows = batch_size, skiprows = i)
            df.columns=columns
            file_name = 'input' + str(i) + '.csv'
            file_list.append(file_name)
            print("[INFO] Creating file: " + file_name)
            df.to_csv(file_name, index=False, header=columns, mode='a', chunksize=batch_size)
        return file_list
    
    def get_subset_data(self, df, ssn_hash, index, n):
        start = end = index
        while(start >= 0 and df.iloc[start]['ssn_hash'] == ssn_hash):
            start = start - 1


        while(end < n and df.iloc[end]['ssn_hash'] == ssn_hash):
            end = end + 1

        return start + 1, end
    
    def process_data(self, file_list):
        file_output = []
        for i in range(len(file_list)):
            file_output.append(self.product_purchase_analysis(file_list[i], "output" + str(i) + ".csv"))
        return file_output
    
    def product_purchase_analysis(self, file_to_read, file_to_write):
        print("[INFO] Doing product_purchase_analysis for: " + file_to_read)
        
        df = pd.read_csv(file_to_read)
        df = df.drop('Unnamed: 0', axis=1)
        n = len(df)
        df['Date'] = pd.to_datetime(df['create_date']).dt.date
        print("[INFO] Length of data: " + str(n))
        

        date=datetime.datetime.now().date()
        for i in range(n):
            start, end = self.get_subset_data(df, df.iloc[i]['ssn_hash'], i, n)
            subset_data = df[start:end]


            date = df.iloc[i]['Date']
            date_next3months = df.iloc[i]['Date'] + relativedelta(months=+3)
            date_next6months = df.iloc[i]['Date'] + relativedelta(months=+6)
            date_next12months = df.iloc[i]['Date'] + relativedelta(months=+12)
            datetime_today = str(datetime.datetime.now().date())
            date_today = pd.to_datetime(datetime_today)

            count_orig = 0
            count_pcl_orig = 0
            count_pl_orig = 0
            count_dep_orig = 0
            count_hm_orig = 0
            count_3 = 0
            count_pcl_3 = 0
            count_pl_3 = 0
            count_dep_3 = 0
            count_hm_3 = 0
            count_6 = 0
            count_pcl_6 = 0
            count_pl_6 = 0
            count_dep_6 = 0
            count_hm_6 = 0
            count_12 = 0
            count_pcl_12 = 0
            count_pl_12 = 0
            count_dep_12 = 0
            count_hm_12 = 0
            count_today = 0
            count_pcl_today = 0
            count_pl_today = 0
            count_dep_today = 0
            count_hm_today = 0

            if start != i:
                subset_df_2 = df[start:i]
                for j in range(len(subset_df_2)):
                    if subset_df_2.iloc[j]['product_type'] == 'PERSONAL_CREDIT_LINE':
                        count_pcl_orig = count_pcl_orig + 1
                    elif subset_df_2.iloc[j]['product_type'] == 'PERSONAL_LOAN':
                        count_pl_orig = count_pl_orig + 1
                    elif subset_df_2.iloc[j]['product_type'] == 'DEPOSIT':
                        count_dep_orig = count_dep_orig + 1
                    elif subset_df_2.iloc[j]['product_type'] == 'HOME_IMPROVEMENT_CREDIT_LINE':
                        count_hm_orig = count_hm_orig + 1
                    else:
                        0

                count_orig = count_pcl_orig + count_pl_orig + count_dep_orig + count_hm_orig




            for j in range(len(subset_data)):
                if date_next3months > subset_data.iloc[j]['Date']:
                    count_3 = count_3 + 1
                    if subset_data.iloc[j]['product_type'] == 'PERSONAL_CREDIT_LINE':
                        count_pcl_3 = count_pcl_3 + 1
                    elif subset_data.iloc[j]['product_type'] == 'PERSONAL_LOAN':
                        count_pl_3 = count_pl_3 + 1
                    elif subset_data.iloc[j]['product_type'] == 'DEPOSIT':
                        count_dep_3 = count_dep_3 + 1
                    elif subset_data.iloc[j]['product_type'] == 'HOME_IMPROVEMENT_CREDIT_LINE':
                        count_hm_3 = count_hm_3 + 1
                    else:
                        0
                if date_next6months > subset_data.iloc[j]['Date']:
                    count_6 = count_6 + 1
                    if subset_data.iloc[j]['product_type'] == 'PERSONAL_CREDIT_LINE':
                        count_pcl_6 = count_pcl_6 + 1
                    elif subset_data.iloc[j]['product_type'] == 'PERSONAL_LOAN':
                        count_pl_6 = count_pl_6 + 1
                    elif subset_data.iloc[j]['product_type'] == 'DEPOSIT':
                        count_dep_6 = count_dep_6 + 1
                    elif subset_data.iloc[j]['product_type'] == 'HOME_IMPROVEMENT_CREDIT_LINE':
                        count_hm_6 = count_hm_6 + 1
                    else:
                        0

                if date_next12months > subset_data.iloc[j]['Date']:
                    count_12 = count_12 + 1
                    if subset_data.iloc[j]['product_type'] == 'PERSONAL_CREDIT_LINE':
                        count_pcl_12 = count_pcl_12 + 1
                    elif subset_data.iloc[j]['product_type'] == 'PERSONAL_LOAN':
                        count_pl_12 = count_pl_12 + 1
                    elif subset_data.iloc[j]['product_type'] == 'DEPOSIT':
                        count_dep_12 = count_dep_12 + 1
                    elif subset_data.iloc[j]['product_type'] == 'HOME_IMPROVEMENT_CREDIT_LINE':
                        count_hm_12 = count_hm_12 + 1
                    else:
                        0

                if datetime_today > subset_data.iloc[j]['create_date']:
                    count_today = count_today + 1
                    if subset_data.iloc[j]['product_type'] == 'PERSONAL_CREDIT_LINE':
                        count_pcl_today = count_pcl_today + 1
                    elif subset_data.iloc[j]['product_type'] == 'PERSONAL_LOAN':
                        count_pl_today = count_pl_today + 1
                    elif subset_data.iloc[j]['product_type'] == 'DEPOSIT':
                        count_dep_today = count_dep_today + 1
                    elif subset_data.iloc[j]['product_type'] == 'HOME_IMPROVEMENT_CREDIT_LINE':
                        count_hm_today = count_hm_today + 1
                    else:
                        0


            df.at[i, 'cnt_tot_post_accts1_orig'] = count_orig
            df.at[i, 'cnt_pcl_post_accts1_orig'] = count_pcl_orig
            df.at[i, 'cnt_pl_post_accts1_orig'] = count_pl_orig
            df.at[i, 'cnt_dep_post_accts1_orig'] = count_dep_orig
            df.at[i, 'cnt_hm_post_accts1_orig'] = count_hm_orig
            df.at[i, 'cnt_tot_post_accts1_3month'] = count_3
            df.at[i, 'cnt_pcl_post_accts1_3month'] = count_pcl_3
            df.at[i, 'cnt_pl_post_accts1_3month'] = count_pl_3
            df.at[i, 'cnt_dep_post_accts1_3month'] = count_dep_3
            df.at[i, 'cnt_hm_post_accts1_3month'] = count_hm_3
            df.at[i, 'cnt_tot_post_accts1_6month'] = count_6
            df.at[i, 'cnt_pcl_post_accts1_6month'] = count_pcl_6
            df.at[i, 'cnt_pl_post_accts1_6month'] = count_pl_6
            df.at[i, 'cnt_dep_post_accts1_6month'] = count_dep_6
            df.at[i, 'cnt_hm_post_accts1_6month'] = count_hm_6
            df.at[i, 'cnt_tot_post_accts1_12month'] = count_12
            df.at[i, 'cnt_pcl_post_accts1_12month'] = count_pcl_12
            df.at[i, 'cnt_pl_post_accts1_12month'] = count_pl_12
            df.at[i, 'cnt_dep_post_accts1_12month'] = count_dep_12
            df.at[i, 'cnt_hm_post_accts1_12month'] = count_hm_12
            df.at[i, 'cnt_tot_post_accts1_today'] = count_today
            df.at[i, 'cnt_pcl_post_accts1_today'] = count_pcl_today
            df.at[i, 'cnt_pl_post_accts1_today'] = count_pl_today
            df.at[i, 'cnt_dep_post_accts1_today'] = count_dep_today
            df.at[i, 'cnt_hm_post_accts1_today'] = count_hm_today
        df = df.drop('create_date', axis=1)
        df.to_csv(file_to_write)
        return file_to_write
    
    
    def merge_output(self, file_list, file_output):
        print("[INFO] Merging the outputs")
        df_list = []
        for i in range(len(file_list)):
            df = pd.read_csv(file_list[i])
            #df['create_date'] =  pd.to_datetime(df['create_date'], infer_datetime_format=True)
            df_list.append(df)
            
        df_big = pd.concat(df_list, ignore_index=True)
        df_big = df_big.drop(['Unnamed: 0'], axis=1)
        
        df_big.to_csv("output_all_products.csv")
        
        df_complete = df_big.loc[df_big['product_type'] == 'PERSONAL_CREDIT_LINE'] 
        df_complete.to_csv(file_output)
        print("[INFO] Merge complete")
        print("[INFO] Created final output file: " + file_output)
        return
    
    


    
        

In [13]:
def main():
    password=input("enter redshift password")
    redshift_file = 'redshift_data.csv'
    secondPCLAnalysis = SecondPCLAnalysis(password)
    secondPCLAnalysis.data_from_redshift(redshift_file)
    file_list = secondPCLAnalysis.file_splitter(redshift_file)
    file_output_list = secondPCLAnalysis.process_data(file_list)
    secondPCLAnalysis.merge_output(file_output_list, "output_pcl.csv")

main()



    
    
    

enter redshift passwordCgblV6kzr8AmQr4oF7gVCX0N
[INFO] Establisging connection with AWS
[INFO] Connection successful
[INFO] Running SQL query


C:\Users\rjangada\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


[INFO] Data received from Redshift
[INFO] Splitting the data from Redshift
[INFO] Creating file: input1.csv
[INFO] Creating file: input6.csv
[INFO] Creating file: input11.csv
[INFO] Doing product_purchase_analysis for: input1.csv
[INFO] Length of data: 5
[INFO] Doing product_purchase_analysis for: input6.csv
[INFO] Length of data: 5
[INFO] Doing product_purchase_analysis for: input11.csv
[INFO] Length of data: 4
[INFO] Merging the outputs
[INFO] Merge complete
[INFO] Created final output file: output_pcl.csv
